In [3]:
import skmob
import pandas as pd
import csv, os, sys, json
import urllib.request
from skmob.preprocessing import clustering


In [38]:
# 주소 정보 전처리해주기
def process_addr(addr_info):
    name_exist = False
    addr = ''
    adm = ''
    roadaddr = ''

    for info in addr_info['result']:
        # 도로명 주소가 있는 경우 건물 이름 빼오기
        if info['name'] == 'roadaddr':
            building_name = info['land']['addition0']['value']
            if len(building_name) == 0:
                road_name = info['land']['name']
                number1 = info['land']['number1']
                number2 = info['land']['number2']
                roadaddr = ' '.join(road_name, number1, number2)
                roadaddr = roadaddr.strip()
            else:
                name_exist = True
                    
        if info['name'] == 'addr':
            area1 = info['region']['area1']
            area2 = info['region']['area2']
            area3 = info['region']['area3']
            area4 = info['region']['area4']
            addr = ' '.join(area1, area2, area3, area4)
            addr = addr.strip()

        if info['name'] == 'admcode':
            area1 = info['region']['area1']
            area2 = info['region']['area2']
            area3 = info['region']['area3']
            area4 = info['region']['area4']
            adm = ' '.join(area1, area2, area3, area4)
            adm = adm.strip()


    if name_exist:
        search = building_name
    else:
        if len(adm) != 0:
            search = adm
        if len(addr) != 0:
            search = addr
        if len(roadaddr) != 0:
            search = roadaddr

    return search

In [39]:
def coords_to_place(x, y):
    client_id = "ktta1uumf3"
    client_secret = "PDFJ27DxqU6bQSsf40GE9DwybUnlgRzZnni42Dqi"
    coords = ','.join([x, y])
    
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=" + \
        coords + "&sourcecrs=epsg:4326&output=json&orders=roadaddr,admcode,roadaddr"

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        addr_info = response_body.decode('utf-8')
        addr_info = json.loads(addr_info)
    else:
        print("Error Code:" + rescode)

    search = process_addr(addr_info)
    
    return search

In [40]:
def place_to_category(place):
    if len(place) == 0 or place == "none":
        return "none"

    client_id = "G3_TXQoFDd0lBFsM8fpG"
    client_secret = "oJaqz0CK0W"
    encText = urllib.parse.quote(place)
    url = "https://openapi.naver.com/v1/search/local?query=" + encText  # JSON 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        category_info = response_body.decode('utf-8')
        category_info = json.loads(category_info)
    else:
        print("Error Code:" + rescode)

    if len(category_info['items']) == 0:
        return "none"
    item = category_info['items'][0]
    category = item['category']

    return category

In [214]:
device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()

for device_id in device_id_list:
    path_dir = './cluster/'+device_id
    file_list = os.listdir(path_dir)

    fields = ['deviceid', 'latitude', 'longitude', 'cluster', 'address']

    for cluster in file_list:
        if not cluster.startswith("cluster_"):
            continue

        result = pd.DataFrame(columns=fields)
        with open('./cluster/'+device_id+'/'+cluster, newline='') as file:
            df = pd.read_csv(file)
            for idx in range (len(df)):
                latitude = df.loc[idx]['latitude']
                longitude = df.loc[idx]['longitude']
                x = "{:.7f}".format(latitude)
                y = "{:.7f}".format(longitude)
                address = coords_to_place(y, x)
                res = pd.DataFrame({
                    'deviceid': [df.loc[idx]['deviceid']],
                    'latitude': [latitude],
                    'longitude': [longitude],
                    'cluster': [df.loc[idx]['cluster']],
                    'address': [address]
                })
                result = pd.concat([result, res])

        c = cluster.split('_')[1].split('.')[0]
        result.to_csv("./cluster/"+device_id+'/address_'+ c +'.csv', index=False)

In [200]:
for device_id in device_id_list:
    path_dir = './cluster/'+device_id
    file_list = os.listdir(path_dir)

    fields = ['deviceid', 'latitude', 'longitude',
              'datetime', 'leaving_datetime', 'cluster', 'category']
    f = open('./cluster/'+device_id+'/category.csv', 'w', newline='')
    write = csv.writer(f)
    write.writerow(fields)

    for cluster in file_list:
        if not cluster.startswith("cluster_"):
            continue
        with open('./cluster/'+device_id+'/'+cluster, newline='') as file:
            df = pd.read_csv(file)
            df['datetime'] = pd.to_datetime(df['datetime'])
            # df['year'] = df['datetime'].map(lambda x: x.isocalendar()[0])
            # df['week'] = df['datetime'].map(lambda x: x.isocalendar()[1])
            latitude = df['latitude'].mean()
            longitude = df['longitude'].mean()

            idx = 0
            dif = 1e6
            for row in range(len(df)):
                tmp = abs(df.loc[row]['latitude']-latitude) + \
                    abs(df.loc[row]['longitude']-longitude)
                if (dif > tmp):
                    idx = row
                    dif = tmp

            x = "{:.7f}".format(df.loc[idx]['latitude'])
            y = "{:.7f}".format(df.loc[idx]['longitude'])
            place = coords_to_place(y, x)
            category = place_to_category(place)

            df['category'] = category
            write.writerow(df.loc[idx])

    f.close()

In [219]:
for device_id in device_id_list:
    df = pd.read_csv('./cluster/'+device_id+'/category.csv')
    df = df.sort_values(by=['cluster'])
    df.to_csv("./cluster/"+device_id+'/category.csv', index=False)

In [26]:
df = pd.read_csv('./cluster/51/address_0.csv')
for i in range(len(df)):
    x = str(df.loc[i]['latitude'])
    y = str(df.loc[i]['longitude'])
    place = coords_to_place(y, x)
    category = place_to_category(place)
    print(place)
    print(category)


{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:21 +0900', 'total': 1, 'start': 1, 'display': 1, 'items': [{'title': '<b>성동구청</b>', 'link': 'http://www.sd.go.kr/', 'category': '공공,사회기관>구청', 'description': '', 'telephone': '', 'address': '서울특별시 성동구 행당동 7', 'roadAddress': '서울특별시 성동구 고산자로 270 성동구청', 'mapx': '315106', 'mapy': '551687'}]}
성동구청 고산자로 270 
공공,사회기관>구청
{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:21 +0900', 'total': 0, 'start': 1, 'display': 0, 'items': []}
유진자동차공업사 고산자로 279 
none
{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:22 +0900', 'total': 0, 'start': 1, 'display': 0, 'items': []}
유진자동차공업사 고산자로 279 
none
{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:22 +0900', 'total': 0, 'start': 1, 'display': 0, 'items': []}
유진자동차공업사 고산자로 279 
none
{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:22 +0900', 'total': 0, 'start': 1, 'display': 0, 'items': []}
 왕십리로20길 16 
none
{'lastBuildDate': 'Sat, 22 Oct 2022 18:05:22 +0900', 'total': 0, 'start': 1, 'display': 0, 'items': []}
 왕십리로20길 16 
none
{'lastBuildDate'

In [37]:
# x = str(37.5646166)
# y = str(127.0356865)
x = str(37.5687204)
y = str(127.0261365)
place = coords_to_place(y,x)
category = place_to_category(place)
# print(place)
print(category)

{'status': {'code': 0, 'name': 'ok', 'message': 'done'}, 'results': [{'name': 'roadaddr', 'code': {'id': '1120010200', 'type': 'L', 'mappingId': '09200102'}, 'region': {'area0': {'name': 'kr', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}, 'area1': {'name': '서울특별시', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 126.9783882, 'y': 37.5666103}}, 'alias': '서울'}, 'area2': {'name': '성동구', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.036821, 'y': 37.563456}}}, 'area3': {'name': '하왕십리동', 'coords': {'center': {'crs': 'EPSG:4326', 'x': 127.02875, 'y': 37.5641583}}}, 'area4': {'name': '', 'coords': {'center': {'crs': '', 'x': 0.0, 'y': 0.0}}}}, 'land': {'type': '', 'number1': '33', 'number2': '', 'addition0': {'type': 'building', 'value': '텐즈힐'}, 'addition1': {'type': 'zipcode', 'value': '04702'}, 'addition2': {'type': 'roadGroupCode', 'value': '112003005033'}, 'addition3': {'type': '', 'value': ''}, 'addition4': {'type': '', 'value': ''}, 'name': '무학로', 'coords': {'center': {'crs